In [1]:

import common

In [2]:
import torch, torchvision
import numpy as np
import matplotlib.pyplot as plt
import pyro
import tqdm
import os,sys
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
from torch.optim import Adam, lr_scheduler


import re, pickle
from torch.utils.data import DataLoader, random_split


In [3]:
# Reproducibility
common.set_seed(1)

In [4]:
layer_type = 'lrt'  # 'bbb' or 'lrt'
activation_type = 'softplus'  # 'softplus' or 'relu'
priors={
    'prior_mu': 0,
    'prior_sigma': 0.1,
    'posterior_mu_initial': (0, 0.1),  # (mean, std) normal_
    'posterior_rho_initial': (-5, 0.1),  # (mean, std) normal_
}
lr_start = 0.001
num_workers = 1
valid_size = 0.2
batch_size = 256
train_ens = 1
valid_ens = 1
beta_type = 0.1  
transform_mnist = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
])
outputs = 10
inputs = 1

In [5]:
trainset = torchvision.datasets.MNIST(root='.', train=True, download=True, transform=transform_mnist)
num_train = len(trainset)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, 
                                           sampler=train_sampler, num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, 
                                           sampler=valid_sampler, num_workers=num_workers)

In [6]:
class BBBLeNet(common.ModuleWrapper):
    '''The architecture of LeNet with Bayesian Layers'''

    def __init__(self, outputs, inputs, priors, layer_type='lrt', activation_type='softplus'):
        super(BBBLeNet, self).__init__()

        self.num_classes = outputs
        self.layer_type = layer_type
        self.priors = priors

        if layer_type=='lrt':
            BBBLinear = common.layers.BBB_LRT_Linear
            BBBConv2d = common.layers.BBB_LRT_Conv2d
        elif layer_type=='bbb':
            BBBLinear = common.layers.BBB_Linear
            BBBConv2d = common.layers.BBB_Conv2d
        else:
            raise ValueError("Undefined layer_type")
        
        if activation_type=='softplus':
            self.act = nn.Softplus
        elif activation_type=='relu':
            self.act = nn.ReLU
        else:
            raise ValueError("Only softplus or relu supported")

        self.conv1 = BBBConv2d(inputs, 6, 5, padding=0, bias=True, priors=self.priors)
        self.act1 = self.act()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = BBBConv2d(6, 16, 5, padding=0, bias=True, priors=self.priors)
        self.act2 = self.act()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.flatten = common.layers.FlattenLayer(5 * 5 * 16)
        self.fc1 = BBBLinear(5 * 5 * 16, 120, bias=True, priors=self.priors)
        self.act3 = self.act()

        self.fc2 = BBBLinear(120, 84, bias=True, priors=self.priors)
        self.act4 = self.act()

        self.fc3 = BBBLinear(84, outputs, bias=True, priors=self.priors)
        
        
class LeNet(nn.Module):

    def __init__(self, outputs, inputs, layer_type='lrt', activation_type='softplus'):
        '''
        Base LeNet model that matches the architecture of BayesianLeNet with randomly 
        initialized weights
        '''
        super(LeNet, self).__init__()
        
        # initialization follows the BBBLeNet initialization, changing
        # BBBLinear and BBBConv2D layers to nn.Linear and nn.Conv2D
        
        if activation_type == 'softplus':
            self.act = nn.Softplus
        elif activation_type == 'relu':
            self.act = nn.ReLU

        self.conv1 = nn.Conv2d(inputs, 6, 5, padding=0, bias=True)
        self.act1 = self.act()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(6, 16, 5, padding=0, bias=True)
        self.act2 = self.act()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(5 * 5 * 16, 120, bias=True)
        self.act3 = self.act()
        self.fc2 = nn.Linear(120, 84, bias=True)
        self.act4 = self.act()
        self.fc3 = nn.Linear(84, outputs, bias=True)


    def sample(self, bbbnet):
        '''
        Takes in a BBBLeNet instance and copies the structure into a LeNet model.
        Replaces the BBBLinear and BBBConv2D that uses sampling in their forward steps
        with regular nn.Linear and nn.Conv2d layers whose weights are initialized by 
        sampling the BBBLeNet model.
        '''    
        ### store activation function used by BNN, only relu and softplus  currently supported
        self.act1 = bbbnet.act()
        self.act2 = bbbnet.act()
        self.act3 = bbbnet.act()
        self.act4 = bbbnet.act()

        ### maxpool
        self.pool1 = nn.MaxPool2d(kernel_size=bbbnet.pool1.kernel_size, stride=bbbnet.pool1.stride)
        self.pool2 = nn.MaxPool2d(kernel_size=bbbnet.pool2.kernel_size, stride=bbbnet.pool2.stride)
        
        ### Create Convolution layers
        self.conv1 = nn.Conv2d(bbbnet.conv1.in_channels, bbbnet.conv1.out_channels, bbbnet.conv1.kernel_size,
                                stride=bbbnet.conv1.stride, padding=bbbnet.conv1.padding, dilation=bbbnet.conv1.dilation,
                                groups=bbbnet.conv1.groups)
        
        self.conv2 = nn.Conv2d(bbbnet.conv2.in_channels, bbbnet.conv2.out_channels, bbbnet.conv2.kernel_size,
                        stride=bbbnet.conv2.stride, padding=bbbnet.conv2.padding, dilation=bbbnet.conv2.dilation,
                        groups=bbbnet.conv2.groups)
        
        # follows the procedure for sampling in the forward methods of BBBConv and 
        # BBBLinearforward to create a fixed set of weights to use for the sampled model

        conv1_W_mu = bbbnet.conv1.W_mu
        conv1_W_rho = bbbnet.conv1.W_rho
        conv1_W_eps = torch.empty(conv1_W_mu.size()).normal_(0,1)
        conv1_W_sigma = torch.log1p(torch.exp(conv1_W_rho))
        conv1_weight = conv1_W_mu + conv1_W_eps * conv1_W_sigma
        if bbbnet.conv1.use_bias:
            conv1_bias_mu = bbbnet.conv1.bias_mu
            conv1_bias_rho = bbbnet.conv1.bias_rho
            conv1_bias_eps = torch.empty(conv1_bias_mu.size()).normal_(0,1)
            conv1_bias_sigma = torch.log1p(torch.exp(conv1_bias_rho))
            conv1_bias = conv1_bias_mu + conv1_bias_eps * conv1_bias_sigma
        else:
            conv1_bias = None
        self.conv1.weight.data = conv1_weight.data
        self.conv1.bias.data = conv1_bias.data


        conv2_W_mu = bbbnet.conv2.W_mu
        conv2_W_rho = bbbnet.conv2.W_rho
        conv2_W_eps = torch.empty(conv2_W_mu.size()).normal_(0,1)
        conv2_W_sigma = torch.log1p(torch.exp(conv2_W_rho))
        conv2_weight = conv2_W_mu + conv2_W_eps * conv2_W_sigma
        if bbbnet.conv2.use_bias:
            conv2_bias_mu = bbbnet.conv2.bias_mu
            conv2_bias_rho = bbbnet.conv2.bias_rho
            conv2_bias_eps = torch.empty(conv2_bias_mu.size()).normal_(0,1)
            conv2_bias_sigma = torch.log1p(torch.exp(conv2_bias_rho))
            conv2_bias = conv2_bias_mu + conv2_bias_eps * conv2_bias_sigma
        else:
            conv2_bias = None
        self.conv2.weight.data = conv2_weight.data
        self.conv2.bias.data = conv2_bias.data
        
        ### Create Linear Layers
        self.fc1 = nn.Linear(bbbnet.fc1.in_features, bbbnet.fc1.out_features, bbbnet.fc1.use_bias)
        self.fc2 = nn.Linear(bbbnet.fc2.in_features, bbbnet.fc2.out_features, bbbnet.fc2.use_bias)
        self.fc3 = nn.Linear(bbbnet.fc3.in_features, bbbnet.fc3.out_features, bbbnet.fc3.use_bias)

        fc1_W_mu = bbbnet.fc1.W_mu
        fc1_W_rho = bbbnet.fc1.W_rho
        fc1_W_eps = torch.empty(fc1_W_mu.size()).normal_(0,1)
        fc1_W_sigma = torch.log1p(torch.exp(fc1_W_rho))
        fc1_weight = fc1_W_mu + fc1_W_eps * fc1_W_sigma
        if bbbnet.fc1.use_bias:
            fc1_bias_mu = bbbnet.fc1.bias_mu
            fc1_bias_rho = bbbnet.fc1.bias_rho
            fc1_bias_eps = torch.empty(fc1_bias_mu.size()).normal_(0,1)
            fc1_bias_sigma = torch.log1p(torch.exp(fc1_bias_rho))
            fc1_bias = fc1_bias_mu + fc1_bias_eps * fc1_bias_sigma
        else:
            fc1_bias = None
        self.fc1.weight.data = fc1_weight.data
        self.fc1.bias.data = fc1_bias.data

        fc2_W_mu = bbbnet.fc2.W_mu
        fc2_W_rho = bbbnet.fc2.W_rho
        fc2_W_eps = torch.empty(fc2_W_mu.size()).normal_(0,1)
        fc2_W_sigma = torch.log1p(torch.exp(fc2_W_rho))
        fc2_weight = fc2_W_mu + fc2_W_eps * fc2_W_sigma
        if bbbnet.fc2.use_bias:
            fc2_bias_mu = bbbnet.fc2.bias_mu
            fc2_bias_rho = bbbnet.fc2.bias_rho
            fc2_bias_eps = torch.empty(fc2_bias_mu.size()).normal_(0,1)
            fc2_bias_sigma = torch.log1p(torch.exp(fc2_bias_rho))
            fc2_bias = fc2_bias_mu + fc2_bias_eps * fc2_bias_sigma
        else:
            fc2_bias = None
        self.fc2.weight.data = fc2_weight.data
        self.fc2.bias.data = fc2_bias.data

        fc3_W_mu = bbbnet.fc3.W_mu
        fc3_W_rho = bbbnet.fc3.W_rho
        fc3_W_eps = torch.empty(fc3_W_mu.size()).normal_(0,1)
        fc3_W_sigma = torch.log1p(torch.exp(fc3_W_rho))
        fc3_weight = fc3_W_mu + fc3_W_eps * fc3_W_sigma
        if bbbnet.fc3.use_bias:
            fc3_bias_mu = bbbnet.fc3.bias_mu
            fc3_bias_rho = bbbnet.fc3.bias_rho
            fc3_bias_eps = torch.empty(fc3_bias_mu.size()).normal_(0,1)
            fc3_bias_sigma = torch.log1p(torch.exp(fc3_bias_rho))
            fc3_bias = fc3_bias_mu + fc3_bias_eps * fc3_bias_sigma
        else:
            fc3_bias = None
        self.fc3.weight.data = fc3_weight.data
        self.fc3.bias.data = fc3_bias.data

        

    def forward(self, x):
        '''
        Forward method follow the order of BayesianLeNet
        '''
        x = self.conv1(x)
        x = self.act1(x)
        x = self.pool1(x)

        x = self.conv2(x)
        x = self.act2(x)
        x = self.pool2(x)

        x = x.view(-1, 5 * 5 * 16)
        x = self.fc1(x)
        x = self.act3(x)
        x = self.fc2(x)
        x = self.act4(x)
        x = self.fc3(x)
        return x

In [7]:
net = BBBLeNet(outputs, inputs, priors, layer_type, activation_type)

In [24]:
def load_models(K = 100, modelname="models/model-cnn.pt"):
    # Load the models
    sampled_models = [LeNet(outputs, inputs, layer_type, activation_type) for i in range(K)]
    for net, state_dict in zip(sampled_models, torch.load(modelname,map_location=torch.device('cpu') )):
        net.load_state_dict(state_dict)
    print("Loaded %d sample models" % K)
    return sampled_models

## Evaluate Test Accuracy (eps=0.05)

### Load Test Sets

In [9]:
transform_back = transforms.Compose([transforms.Resize((28,28))])

In [12]:
dirs = [d for d in os.listdir("mnist_adv_CNN_itcm_0.05/") if "test_images_med" in d]

images  = []
targets = []
for d in dirs:
    with open("mnist_adv_CNN_itcm_0.05/" + d, 'rb') as handle:
        temp = pickle.load(handle)
        images.append(temp["images"])
        targets.append(temp["labels"])
        
images  = torch.vstack(images)
images  = transform_back(images)
# images  = images[:,0,:,:]

targets = torch.hstack(targets)

# Test dataset
test_dataset = torchvision.datasets.MNIST('.', train=False, download=True,
                       transform=transform_mnist)

# Test data loader with batch_size 128
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=True,
                                         generator=torch.Generator().manual_seed(156))

# AdvTest dataset
adv_test_dataset = torchvision.datasets.MNIST('.', train=False, download=True,
                       transform=transform_mnist)
adv_test_dataset.data    = None
adv_test_dataset.targets = None

adv_test_dataset.data    = images
adv_test_dataset.targets = targets

# Test data loader with batch_size 1
adv_test_loader = torch.utils.data.DataLoader(adv_test_dataset, batch_size=128, shuffle=True,
                                         generator=torch.Generator().manual_seed(156))

In [17]:
dirs = [d for d in os.listdir("mnist_adv_CNN_itcm_0.05/") if "test_images_champ" in d]

images  = []
targets = []
for d in dirs:
    with open("mnist_adv_CNN_itcm_0.05/" + d, 'rb') as handle:
        temp = pickle.load(handle)
        images.append(temp["images"])
        targets.append(temp["labels"])
        
images  = torch.vstack(images)
images  = transform_back(images)
# images  = images[:,0,:,:]
targets = torch.hstack(targets)

# AdvTest dataset
adv_champ_test_dataset = torchvision.datasets.MNIST('.', train=False, download=True,
                       transform=transform_mnist)
adv_champ_test_dataset.data    = None
adv_champ_test_dataset.targets = None

adv_champ_test_dataset.data    = images
adv_champ_test_dataset.targets = targets

# Test data loader with batch_size 1
adv_champ_test_loader = torch.utils.data.DataLoader(adv_champ_test_dataset, batch_size=128, shuffle=True,
                                         generator=torch.Generator().manual_seed(156))

In [18]:
dirs = [d for d in os.listdir("mnist_adv_CNN_itcm_0.05/") if "test_images_mean" in d]

images  = []
targets = []
for d in dirs:
    with open("mnist_adv_CNN_itcm_0.05/" + d, 'rb') as handle:
        temp = pickle.load(handle)
        images.append(temp["images"])
        targets.append(temp["labels"])
        
images  = torch.vstack(images)
images  = transform_back(images)
# images  = images[:,0,:,:]
targets = torch.hstack(targets)

# AdvTest dataset
adv_mean_test_dataset = torchvision.datasets.MNIST('.', train=False, download=True,
                       transform=transform_mnist)
adv_mean_test_dataset.data    = None
adv_mean_test_dataset.targets = None

adv_mean_test_dataset.data    = images
adv_mean_test_dataset.targets = targets

# Test data loader with batch_size 1
adv_mean_test_loader = torch.utils.data.DataLoader(adv_mean_test_dataset, batch_size=128, shuffle=True,
                                         generator=torch.Generator().manual_seed(156))

In [19]:
dirs = [d for d in os.listdir("mnist_adv_CNN_itcm_0.18/") if "test_images_med" in d]

images  = []
targets = []
for d in dirs:
    with open("mnist_adv_CNN_itcm_0.18/" + d, 'rb') as handle:
        temp = pickle.load(handle)
        images.append(temp["images"])
        targets.append(temp["labels"])
        
images  = torch.vstack(images)
images  = transform_back(images)
# images  = images[:,0,:,:]

targets = torch.hstack(targets)

# Test dataset
test_dataset = torchvision.datasets.MNIST('.', train=False, download=True,
                       transform=transform_mnist)

# Test data loader with batch_size 128
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=True,
                                         generator=torch.Generator().manual_seed(156))

# AdvTest dataset
adv_test_18_dataset = torchvision.datasets.MNIST('.', train=False, download=True,
                       transform=transform_mnist)
adv_test_18_dataset.data    = None
adv_test_18_dataset.targets = None

adv_test_18_dataset.data    = images
adv_test_18_dataset.targets = targets

# Test data loader with batch_size 1
adv_test_18_loader = torch.utils.data.DataLoader(adv_test_18_dataset, batch_size=128, shuffle=True,
                                         generator=torch.Generator().manual_seed(156))

In [21]:
dirs = [d for d in os.listdir("mnist_adv_CNN_itcm_0.18/") if "test_images_champ" in d]

images  = []
targets = []
for d in dirs:
    with open("mnist_adv_CNN_itcm_0.18/" + d, 'rb') as handle:
        temp = pickle.load(handle)
        images.append(temp["images"])
        targets.append(temp["labels"])
        
images  = torch.vstack(images)
images  = transform_back(images)
# images  = images[:,0,:,:]

targets = torch.hstack(targets)

# Test dataset
test_dataset = torchvision.datasets.MNIST('.', train=False, download=True,
                       transform=transform_mnist)

# Test data loader with batch_size 128
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=True,
                                         generator=torch.Generator().manual_seed(156))

# AdvTest dataset
adv_champ_test_18_dataset = torchvision.datasets.MNIST('.', train=False, download=True,
                       transform=transform_mnist)
adv_champ_test_18_dataset.data    = None
adv_champ_test_18_dataset.targets = None

adv_champ_test_18_dataset.data    = images
adv_champ_test_18_dataset.targets = targets

# Test data loader with batch_size 1
adv_champ_test_18_loader = torch.utils.data.DataLoader(adv_champ_test_18_dataset, batch_size=128, shuffle=True,
                                         generator=torch.Generator().manual_seed(156))

In [22]:
dirs = [d for d in os.listdir("mnist_adv_CNN_itcm_0.18/") if "test_images_mean" in d]

images  = []
targets = []
for d in dirs:
    with open("mnist_adv_CNN_itcm_0.18/" + d, 'rb') as handle:
        temp = pickle.load(handle)
        images.append(temp["images"])
        targets.append(temp["labels"])
        
images  = torch.vstack(images)
images  = transform_back(images)
# images  = images[:,0,:,:]

targets = torch.hstack(targets)

# Test dataset
test_dataset = torchvision.datasets.MNIST('.', train=False, download=True,
                       transform=transform_mnist)

# Test data loader with batch_size 128
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=True,
                                         generator=torch.Generator().manual_seed(156))

# AdvTest dataset
adv_mean_test_18_dataset = torchvision.datasets.MNIST('.', train=False, download=True,
                       transform=transform_mnist)
adv_mean_test_18_dataset.data    = None
adv_mean_test_18_dataset.targets = None

adv_mean_test_18_dataset.data    = images
adv_mean_test_18_dataset.targets = targets

# Test data loader with batch_size 1
adv_mean_test_18_loader = torch.utils.data.DataLoader(adv_mean_test_18_dataset, batch_size=128, shuffle=True,
                                         generator=torch.Generator().manual_seed(156))

In [57]:
dirs = [d for d in os.listdir("mnist_adv_CNN_itcm_0.5/") if "test_images_med" in d]

images  = []
targets = []
for d in dirs:
    with open("mnist_adv_CNN_itcm_0.5/" + d, 'rb') as handle:
        temp = pickle.load(handle)
        images.append(temp["images"])
        targets.append(temp["labels"])
        
images  = torch.vstack(images)
images  = transform_back(images)
# images  = images[:,0,:,:]

targets = torch.hstack(targets)

# Test dataset
test_dataset = torchvision.datasets.MNIST('.', train=False, download=True,
                       transform=transform_mnist)

# Test data loader with batch_size 128
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=True,
                                         generator=torch.Generator().manual_seed(156))

# AdvTest dataset
adv_test_50_dataset = torchvision.datasets.MNIST('.', train=False, download=True,
                       transform=transform_mnist)
adv_test_50_dataset.data    = None
adv_test_50_dataset.targets = None

adv_test_50_dataset.data    = images
adv_test_50_dataset.targets = targets

# Test data loader with batch_size 1
adv_test_50_loader = torch.utils.data.DataLoader(adv_test_50_dataset, batch_size=128, shuffle=True,
                                         generator=torch.Generator().manual_seed(156))

In [58]:
dirs = [d for d in os.listdir("mnist_adv_CNN_itcm_0.5/") if "test_images_champ" in d]

images  = []
targets = []
for d in dirs:
    with open("mnist_adv_CNN_itcm_0.5/" + d, 'rb') as handle:
        temp = pickle.load(handle)
        images.append(temp["images"])
        targets.append(temp["labels"])
        
images  = torch.vstack(images)
images  = transform_back(images)
# images  = images[:,0,:,:]

targets = torch.hstack(targets)

# Test dataset
test_dataset = torchvision.datasets.MNIST('.', train=False, download=True,
                       transform=transform_mnist)

# Test data loader with batch_size 128
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=True,
                                         generator=torch.Generator().manual_seed(156))

# AdvTest dataset
adv_champ_test_50_dataset = torchvision.datasets.MNIST('.', train=False, download=True,
                       transform=transform_mnist)
adv_champ_test_50_dataset.data    = None
adv_champ_test_50_dataset.targets = None

adv_champ_test_50_dataset.data    = images
adv_champ_test_50_dataset.targets = targets

# Test data loader with batch_size 1
adv_champ_test_50_loader = torch.utils.data.DataLoader(adv_champ_test_50_dataset, batch_size=128, shuffle=True,
                                         generator=torch.Generator().manual_seed(156))

### Evaluate

In [26]:
img = next(iter(test_loader))

In [27]:
img[0].shape

torch.Size([128, 1, 32, 32])

In [29]:
def evaluate_accuracy(model, loader):
    total_correct = 0
    for data in loader:
        images, labels = data
        # images = images.view(-1, 28*28)
        y    = model(images)
        pred = torch.argmax(y, dim=1)
        total_correct += torch.sum(pred==labels).item()
    return total_correct / len(loader.dataset.targets)

### AdvBNN vs BNN

In [69]:
sampled_adv_models_50       = load_models(K = 50,modelname="models/lenet-mnist-adv-med-0.5.pt")
sampled_adv_models_50_champ = load_models(K = 50,modelname="models/lenet-mnist-adv-champ-0.5.pt")
# sampled_adv_models_50_mean  = load_models(K = 50,modelname="models/model-cnn-adv-mean0.50.pt")

Loaded 50 sample models
Loaded 50 sample models


In [47]:
sampled_adv_models_18       = load_models(K = 50,modelname="models/lenet-mnist-adv-med-0.18.pt")
sampled_adv_models_18_champ = load_models(K = 50,modelname="models/lenet-mnist-adv-champ-0.18.pt")
# sampled_adv_models_18_mean  = load_models(K = 50,modelname="models/model-cnn-adv-mean0.18.pt")

Loaded 50 sample models
Loaded 50 sample models


In [46]:
sampled_adv_models_05       = load_models(K = 50,modelname="models/lenet-mnist-adv-med-0.05.pt")
sampled_adv_models_05_champ = load_models(K = 50,modelname="models/lenet-mnist-adv-champ-0.05.pt")
# sampled_adv_models_05_mean  = load_models(K = 50,modelname="models/model-cnn-adv-mean0.05.pt")

sampled_models        = load_models(K = 50,modelname="models/model-cnn.pt")

Loaded 50 sample models
Loaded 50 sample models
Loaded 50 sample models


In [27]:
cnn_model = torch.load("models/CNN.pt")

#### CNN vs Test

In [30]:
evaluate_accuracy(cnn_model, test_loader)

0.9827

In [31]:
evaluate_accuracy(cnn_model, adv_test_loader)

0.7455830388692579

In [32]:
evaluate_accuracy(cnn_model, adv_champ_test_loader)

0.7425930959499865

In [33]:
evaluate_accuracy(cnn_model, adv_mean_test_loader)

0.7633867898885567

In [41]:
evaluate_accuracy(cnn_model, adv_test_18_loader)

0.8604706160577037

In [42]:
evaluate_accuracy(cnn_model, adv_champ_test_18_loader)

0.8035946553151235

In [59]:
evaluate_accuracy(cnn_model, adv_test_50_loader)

0.4835397835063051

In [61]:
evaluate_accuracy(cnn_model, adv_champ_test_50_loader)

0.2807722352416025

#### AdvBNN(eps=0.05) vs Test

In [48]:
accs = [evaluate_accuracy(model, test_loader) for model in sampled_adv_models_05]
print("max:  ",max(accs))
print("mean: ", np.mean(accs), ", sd: ", round(np.std(accs),4))
print("min:  ",min(accs))

max:   0.9891
mean:  0.985804 , sd:  0.002
min:   0.9804


#### AdvBNN_champ(eps=0.05) vs Test

In [34]:
accs = [evaluate_accuracy(model, test_loader) for model in sampled_adv_models_05_champ]
print("max:  ",max(accs))
print("mean: ", np.mean(accs), ", sd: ", round(np.std(accs),4))
print("min:  ",min(accs))

max:   0.989
mean:  0.985404 , sd:  0.002
min:   0.9804


#### AdvBNN(eps=18) vs Test

In [49]:
accs = [evaluate_accuracy(model, test_loader) for model in sampled_adv_models_18]
print("max: ",max(accs))
print("mean: ", np.mean(accs), ", sd: ", round(np.std(accs),4))
print("min: ",min(accs))

max:  0.9885
mean:  0.9853299999999997 , sd:  0.002
min:  0.979


#### AdvBNN_champ(eps=18) vs Test

In [36]:
accs = [evaluate_accuracy(model, test_loader) for model in sampled_adv_models_18_champ]
print("max: ",max(accs))
print("mean: ", np.mean(accs), ", sd: ", round(np.std(accs),4))
print("min: ",min(accs))

max:  0.9887
mean:  0.985048 , sd:  0.002
min:  0.9807


#### AdvBNN(eps=50) vs Test

In [62]:
accs = [evaluate_accuracy(model, test_loader) for model in sampled_adv_models_50]
print("max: ",max(accs))
print("mean: ", np.mean(accs), ", sd: ", round(np.std(accs),4))
print("min: ",min(accs))

max:  0.9876
mean:  0.984232 , sd:  0.0017
min:  0.98


#### AdvBNN_champ(eps=50) vs Test

In [70]:
accs = [evaluate_accuracy(model, test_loader) for model in sampled_adv_models_50_champ]
print("max: ",max(accs))
print("mean: ", np.mean(accs), ", sd: ", round(np.std(accs),4))
print("min: ",min(accs))

max:  0.9869
mean:  0.984666 , sd:  0.0017
min:  0.9793


#### BNN vs Test

In [41]:
accs = [evaluate_accuracy(model, test_loader) for model in sampled_models]
print("max: ",max(accs))
print("mean: ", np.mean(accs), ", sd: ", round(np.std(accs),4))
print("min: ",min(accs))

max:  0.9824
mean:  0.975064 , sd:  0.0083
min:  0.938


#### AdvBNN(eps=0.05) vs Adv Test

In [50]:
accs = [evaluate_accuracy(model, adv_test_loader) for model in  sampled_adv_models_05]
print("max:  ",max(accs))
print("mean: ", np.mean(accs), ", sd: ", round(np.std(accs),4))
print("min:  ",min(accs))

max:   0.8594726827942376
mean:  0.8438570263658604 , sd:  0.0083
min:   0.8240010872519706


#### AdvBNN_champ(eps=0.05) vs AdvTest

In [37]:
accs = [evaluate_accuracy(model, adv_test_loader) for model in sampled_adv_models_05_champ]
print("max:  ",max(accs))
print("mean: ", np.mean(accs), ", sd: ", round(np.std(accs),4))
print("min:  ",min(accs))

max:   0.856754552867627
mean:  0.8432128295732536 , sd:  0.0074
min:   0.8253601522152759


#### AdvBNN(eps=0.18) vs Adv Test

In [51]:
accs = [evaluate_accuracy(model, adv_test_loader) for model in  sampled_adv_models_18]
print("max:  ",max(accs))
print("mean: ", np.mean(accs), ", sd: ", round(np.std(accs),4))
print("min:  ",min(accs))

max:   0.8547159554226692
mean:  0.8410138624626258 , sd:  0.0083
min:   0.820195705354716


#### AdvBNN_champ(eps=0.18) vs Adv Test

In [38]:
accs = [evaluate_accuracy(model, adv_test_loader) for model in  sampled_adv_models_18_champ]
print("max:  ",max(accs))
print("mean: ", np.mean(accs), ", sd: ", round(np.std(accs),4))
print("min:  ",min(accs))

max:   0.856482739874966
mean:  0.8421473226420223 , sd:  0.0084
min:   0.8206034248437075


#### AdvBNN(eps=0.50) vs Adv Test

In [64]:
accs = [evaluate_accuracy(model, adv_test_loader) for model in  sampled_adv_models_50]
print("max:  ",max(accs))
print("mean: ", np.mean(accs), ", sd: ", round(np.std(accs),4))
print("min:  ",min(accs))

max:   0.8151671649904866
mean:  0.7963468333786355 , sd:  0.0118
min:   0.7628431639032346


#### AdvBNN_champ(eps=0.50) vs Adv Test

In [71]:
accs = [evaluate_accuracy(model, adv_test_loader) for model in  sampled_adv_models_50_champ]
print("max:  ",max(accs))
print("mean: ", np.mean(accs), ", sd: ", round(np.std(accs),4))
print("min:  ",min(accs))

max:   0.8101386246262572
mean:  0.7832780646914922 , sd:  0.0149
min:   0.7519706441967926


#### BNN vs. Adv Test

In [39]:
accs = [evaluate_accuracy(model, adv_test_loader) for model in  sampled_models]
print("max:  ",max(accs))
print("mean: ", np.mean(accs), ", sd: ", round(np.std(accs),4))
print("min:  ",min(accs))

max:   0.7606686599619462
mean:  0.6968578418048382 , sd:  0.0376
min:   0.5863006251698831


In [40]:
accs = [evaluate_accuracy(model, adv_champ_test_loader) for model in  sampled_models]
print("max:  ",max(accs))
print("mean: ", np.mean(accs), ", sd: ", round(np.std(accs),4))
print("min:  ",min(accs))

max:   0.7590377820059799
mean:  0.6969611307420496 , sd:  0.0384
min:   0.5830388692579506


In [43]:
accs = [evaluate_accuracy(model, adv_test_18_loader) for model in  sampled_models]
print("max:  ",max(accs))
print("mean: ", np.mean(accs), ", sd: ", round(np.std(accs),4))
print("min:  ",min(accs))

max:   0.9065862599030389
mean:  0.8366371053565093 , sd:  0.0414
min:   0.7172756296559063


### Adversarial Test (eps=0.18)

#### CNN vs Test

In [41]:
evaluate_accuracy(cnn_model, adv_test_18_loader)

0.5607749192792417

#### AdvBNN(eps=0.05) vs Adv Test (eps=0.18)

In [52]:
accs = [evaluate_accuracy(model, adv_test_18_loader) for model in  sampled_adv_models_05]
print("max:  ",max(accs))
print("mean: ", np.mean(accs), ", sd: ", round(np.std(accs),4))
print("min:  ",min(accs))

max:   0.9495092822513894
mean:  0.92960151353908 , sd:  0.0147
min:   0.8823459855740806


#### AdvBNN_champ(eps=0.05) vs AdvTest

In [53]:
accs = [evaluate_accuracy(model, adv_test_18_loader) for model in sampled_adv_models_05_champ]
print("max:  ",max(accs))
print("mean: ", np.mean(accs), ", sd: ", round(np.std(accs),4))
print("min:  ",min(accs))

max:   0.9509282251389382
mean:  0.9333735367151473 , sd:  0.0142
min:   0.8947617358401324


#### AdvBNN(eps=0.18) vs Adv Test

In [54]:
accs = [evaluate_accuracy(model, adv_test_18_loader) for model in  sampled_adv_models_18]
print("max:  ",max(accs))
print("mean: ", np.mean(accs), ", sd: ", round(np.std(accs),4))
print("min:  ",min(accs))

max:   0.969019746955185
mean:  0.9635544519333098 , sd:  0.0035
min:   0.9498640179732766


#### AdvBNN_champ(eps=0.50) vs Adv Test

In [72]:
accs = [evaluate_accuracy(model, adv_test_18_loader) for model in  sampled_adv_models_50_champ]
print("max:  ",max(accs))
print("mean: ", np.mean(accs), ", sd: ", round(np.std(accs),4))
print("min:  ",min(accs))

max:   0.9606243348705215
mean:  0.9483883173702259 , sd:  0.0066
min:   0.9316542509164006


#### AdvBNN(eps=0.50) vs Adv Test

In [73]:
accs = [evaluate_accuracy(model, adv_test_18_loader) for model in  sampled_adv_models_50]
print("max:  ",max(accs))
print("mean: ", np.mean(accs), ", sd: ", round(np.std(accs),4))
print("min:  ",min(accs))

max:   0.9620432777580702
mean:  0.9547853848882581 , sd:  0.0048
min:   0.9413503606479839


#### AdvBNN_champ(eps=0.18) vs Adv Test

In [45]:
accs = [evaluate_accuracy(model, adv_test_18_loader) for model in  sampled_adv_models_18_champ]
print("max:  ",max(accs))
print("mean: ", np.mean(accs), ", sd: ", round(np.std(accs),4))
print("min:  ",min(accs))

max:   0.9705569350833629
mean:  0.9659571952228923 , sd:  0.0023
min:   0.9588506562610855


#### BNN vs. Adv Test

In [55]:
accs = [evaluate_accuracy(model, adv_test_18_loader) for model in  sampled_models]
print("max:  ",max(accs))
print("mean: ", np.mean(accs), ", sd: ", round(np.std(accs),4))
print("min:  ",min(accs))

max:   0.9065862599030389
mean:  0.8366371053565093 , sd:  0.0414
min:   0.7172756296559063


accs = [evaluate_accuracy(model, adv_champ_test_18_loader) for model in  sampled_models]
print("max:  ",max(accs))
print("mean: ", np.mean(accs), ", sd: ", round(np.std(accs),4))
print("min:  ",min(accs))

### Adversarial Test (eps=0.50)

#### CNN vs Test

In [ ]:
evaluate_accuracy(cnn_model, adv_test_50_loader)

#### AdvBNN(eps=0.05) vs Adv Test (eps=0.50)

In [74]:
accs = [evaluate_accuracy(model, adv_test_50_loader) for model in  sampled_adv_models_05]
print("max:  ",max(accs))
print("mean: ", np.mean(accs), ", sd: ", round(np.std(accs),4))
print("min:  ",min(accs))

max:   0.5594241714094409
mean:  0.44562214038611764 , sd:  0.046
min:   0.33433768552616894


#### AdvBNN(eps=0.05) vs Adv Test (Champ)

accs = [evaluate_accuracy(model, adv_champ_test_loader) for model in  sampled_adv_models_05]
print("max:  ",max(accs))
print("mean: ", np.mean(accs), ", sd: ", round(np.std(accs),4))
print("min:  ",min(accs))

#### AdvBNN_champ(eps=0.05) vs AdvTest

In [75]:
accs = [evaluate_accuracy(model, adv_test_50_loader) for model in sampled_adv_models_05_champ]
print("max:  ",max(accs))
print("mean: ", np.mean(accs), ", sd: ", round(np.std(accs),4))
print("min:  ",min(accs))

max:   0.5566343042071198
mean:  0.437580627162147 , sd:  0.0511
min:   0.3020868206673362


#### AdvBNN(eps=0.18) vs Adv Test

In [76]:
accs = [evaluate_accuracy(model, adv_test_50_loader) for model in  sampled_adv_models_18]
print("max:  ",max(accs))
print("mean: ", np.mean(accs), ", sd: ", round(np.std(accs),4))
print("min:  ",min(accs))

max:   0.7084030800133914
mean:  0.6112554402410446 , sd:  0.0507
min:   0.5181341368150876


#### AdvBNN_champ(eps=0.18) vs Adv Test

In [77]:
accs = [evaluate_accuracy(model, adv_test_50_loader) for model in  sampled_adv_models_18_champ]
print("max:  ",max(accs))
print("mean: ", np.mean(accs), ", sd: ", round(np.std(accs),4))
print("min:  ",min(accs))

max:   0.7733511884834282
mean:  0.6913223970539002 , sd:  0.0426
min:   0.5975895547371946


#### AdvBNN(eps=0.50) vs Adv Test

In [78]:
accs = [evaluate_accuracy(model, adv_test_50_loader) for model in  sampled_adv_models_50]
print("max:  ",max(accs))
print("mean: ", np.mean(accs), ", sd: ", round(np.std(accs),4))
print("min:  ",min(accs))

max:   0.9631737529293606
mean:  0.9531123758509092 , sd:  0.0076
min:   0.9235576386564


#### AdvBNN_champ(eps=0.50) vs Adv Test

In [79]:
accs = [evaluate_accuracy(model, adv_test_50_loader) for model in  sampled_adv_models_50_champ]
print("max:  ",max(accs))
print("mean: ", np.mean(accs), ", sd: ", round(np.std(accs),4))
print("min:  ",min(accs))

max:   0.9569244503961611
mean:  0.9462359111706283 , sd:  0.007
min:   0.9192054458207789


#### BNN vs. Adv Test

In [80]:
accs = [evaluate_accuracy(model, adv_test_50_loader) for model in  sampled_models]
print("max:  ",max(accs))
print("mean: ", np.mean(accs), ", sd: ", round(np.std(accs),4))
print("min:  ",min(accs))

max:   0.3740653944872224
mean:  0.26301082468474496 , sd:  0.0471
min:   0.1648253543131347
